In [46]:
from agent_runner import AgentRunner

from events import (
    AgentTextStream,
    ToolCallsOutputStart,
    ToolCallsOutput,
    AgentHandoff,
    ToolCallResult,
    AgentTextOutput,
    AgentResult,
    EventType,
    StructuredOutput
)

import time

from pydantic import BaseModel
from pydantic_ai import Agent

from voice_agent_flow.llms import create_pydantic_azure_openai

from pydantic_ai.messages import (
    UserPromptPart, 
    TextPart, 
    ToolCallPart, 
    ToolReturnPart, 
    SystemPromptPart
)

from pydantic_ai import ModelRequest, ModelResponse
from datetime import datetime

from dataclasses import dataclass

class PydanticMessageAdaptor:

    def user(
        self,
        content:str, 
        timestamp: datetime = None) -> ModelRequest:
        
        if timestamp is None:
            timestamp = datetime.now()
            
        return ModelRequest(
            parts = [
                UserPromptPart(content=content, timestamp=timestamp)
            ]
        )
        
    def assistant(
        self,
        content:str,
        timestamp = None) -> ModelResponse:
        
        if timestamp is None:
            timestamp = datetime.now()
        
        return ModelResponse(
            parts = [TextPart(content=content)],
            timestamp=timestamp
        )
        
    def tool_call(
        self,
        tool_name:str, 
        args: str, 
        tool_call_id:str = 'fake', 
        timestamp = None) -> ModelResponse:
        
        if timestamp is None:
            timestamp = datetime.now()
        
        return ModelResponse(
            parts = [ToolCallPart(
                tool_name=tool_name, 
                args=args, 
                tool_call_id=tool_call_id)],
            timestamp=timestamp
        )
        
    def tool_return(
        self,
        tool_name:str, 
        content:str,
        tool_call_id:str = 'fake', 
        timestamp = None) -> ModelResponse:
        
        if timestamp is None:
            timestamp = datetime.now()
            
        return ModelRequest(
            parts = [ToolReturnPart(
                tool_name=tool_name, 
                content=content, 
                tool_call_id=tool_call_id, 
                timestamp=timestamp)]
        )
        
    def system(
        self, 
        content:str, 
        timestamp = None) -> ModelRequest:
        
        if timestamp is None:
            timestamp = datetime.now()
            
        return ModelRequest(
            parts = [
                SystemPromptPart(content=content, timestamp=timestamp)
            ]
        )
        
    def auto(self, msg:dict):
        
        if 'timestamp' in msg:
            timestamp = datetime.fromisoformat(msg['timestamp'])
            
        else:
            timestamp = datetime.now()
            
        if msg['role'] == 'system':
            return self.system(content=msg['content'], timestamp=timestamp)
        
        elif msg['role'] == 'user':
            return self.user(content=msg['content'], timestamp=timestamp)
        
        elif msg['role'] == 'tool':
            return self.tool_return(
                tool_name=msg['tool_name'], 
                content=msg['content'],
                tool_call_id=msg.get('tool_call_id', 'fake'),
                timestamp=timestamp
            )
            
        elif msg['role'] == 'assistant':
            if "content" in msg:
                return self.assistant(content=msg['content'], timestamp=timestamp)
            
            elif "tool_name" in msg:
                return self.tool_call(
                    tool_name=msg['tool_name'], 
                    args=msg.get('args', ''), 
                    tool_call_id=msg.get('tool_call_id', 'fake'), 
                    timestamp=timestamp
                )
            else:
                raise ValueError(f"Invalid assistant message format: {msg}")
        else:
            raise ValueError(f"Unknown role: {msg['role']}")
    
    def validate(self, message_history: list):
        
        history = [self.auto(msg) for msg in message_history]
        
        merged_history = []
        for msg in history:
            if not merged_history:
                merged_history.append(msg)
                
            else:
                last_msg = merged_history[-1]
                if isinstance(last_msg, ModelRequest) and isinstance(msg, ModelRequest):
                    last_msg.parts.extend(msg.parts)
                    
                elif isinstance(last_msg, ModelResponse) and isinstance(msg, ModelResponse):
                    last_msg.parts.extend(msg.parts)
                    
                else:
                    merged_history.append(msg)
                    
        return merged_history
            
    
start = time.time()
pmsg = PydanticMessageAdaptor()

pmsg.tool_return('search', 'search result here')
pmsg.assistant('This is a response from the assistant.')
pmsg.user('What is the weather today?')
pmsg.tool_call('search', 'weather today')
pmsg.system("You are a helpful assistant.")
end = time.time()

print(f"Time taken: {end - start:.4f} seconds")


message_history = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather today?"},
    {"role": "assistant", "tool_name": "search", "args": "weather today", "tool_call_id": "1"},
    {"role": "tool", "tool_name": "search", "content": "The weather today is sunny with a high of 25°C.", "tool_call_id": "1"},
    {"role": "assistant", "content": "The weather today is sunny with a high of 25°C."},
    {'role': "user", "content": "Let me introduce my self, My name is Wang Huan and I am 32 years old, a middle age AI engineer living in the city of Beijing."}
]

pmsg.validate(message_history)

Time taken: 0.0001 seconds


[ModelRequest(parts=[SystemPromptPart(content='You are a helpful assistant.', timestamp=datetime.datetime(2026, 2, 21, 1, 35, 52, 403601)), UserPromptPart(content='What is the weather today?', timestamp=datetime.datetime(2026, 2, 21, 1, 35, 52, 403606))]),
 ModelResponse(parts=[ToolCallPart(tool_name='search', args='weather today', tool_call_id='1')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 21, 1, 35, 52, 403608)),
 ModelRequest(parts=[ToolReturnPart(tool_name='search', content='The weather today is sunny with a high of 25°C.', tool_call_id='1', timestamp=datetime.datetime(2026, 2, 21, 1, 35, 52, 403614))]),
 ModelResponse(parts=[TextPart(content='The weather today is sunny with a high of 25°C.')], usage=RequestUsage(), timestamp=datetime.datetime(2026, 2, 21, 1, 35, 52, 403617)),
 ModelRequest(parts=[UserPromptPart(content='Let me introduce my self, My name is Wang Huan and I am 32 years old, a middle age AI engineer living in the city of Beijing.', timestamp=dateti

In [6]:
class Person(BaseModel):
    name:str
    age:int
    
    def transfer(self):
        print("\n⚾️ I want to transfer to the next step.")
    
    
def weather_tool(location: str, credential = "3234980988908333498") -> str:
    """Search the weather for a location."""
    return f"The weather in {location} is snowing, the temperature will be below -5 degrees Celsius."

tool_call_query = 'What is the weather in New York City?'
name_query = "My name is wanghuan, 32 years old."


model = create_pydantic_azure_openai(model_name = "gpt-4.1")

# this is the actual agent definition.
agent = Agent(
    model, 
    tools = [weather_tool],
    instructions = (
    "You are a helpful assistant. you chat with users friendly."
    "When user asks questions about weather, do call the weather tool to get the weather information, and give suggestions on clothing based on the weather. "
    "When user mentions any person with name and age, you extract the name and age, return a json object defined by `Person` schema."
    "Reply in Chinese."
    ""),
    output_type = str | Person)

runner = AgentRunner(agent)

async def run_one_turn(query):
    print(f"\n=== Running query: {query} ===")
    no_text = True
    
    async for event in runner.run(query):
        
        # handle structured output events.
        if not isinstance(event, AgentResult):
            continue
        
        # add text stream to context / pending responses
        if isinstance(event.event, AgentTextStream):
            time.sleep(0.05)
            if event.event.delta is None or event.event.delta == "":
                continue
            
            if no_text:
                print("\n⚾️ Agent starts to generate text:")
            print(event.event.delta, end='', flush=True)
            no_text = False
            continue
        
        if isinstance(event.event, StructuredOutput):
            print("\n⚾️ Found structured output:")
            print(event.event.message)
            if hasattr(event.event.message, 'transfer'):
                event.event.message.transfer()
        
        # add tool history to memory
        else:
            print("\n⚾️ New Event:")
            print(event)
        
print("\n=== Test tool call query ===")
await run_one_turn(tool_call_query)

print("\n\n=== Test name query ===")
await run_one_turn(name_query)


=== Test tool call query ===

=== Running query: What is the weather in New York City? ===


INFO:httpx:HTTP Request: POST https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:root:呃, 稍等我想下啊。



⚾️ New Event:
AgentResult(event=ToolCallsOutputStart(status=None, message={'tool_name': 'weather_tool', 'args': '', 'tool_call_id': 'call_QjTaAVBOSRt87bZfDHhWWYLk', 'id': None, 'provider_details': None, 'part_kind': 'tool-call'}), event_type='ToolCallsOutputStart', finish_reason='', last_agent_name='')

⚾️ New Event:
AgentResult(event=ToolCallsOutput(status=None, message={'tool_name': 'weather_tool', 'args': '{"location":"New York City"}', 'tool_call_id': 'call_QjTaAVBOSRt87bZfDHhWWYLk', 'id': None, 'provider_details': None, 'part_kind': 'tool-call'}), event_type='ToolCallsOutput', finish_reason='', last_agent_name='')

⚾️ New Event:
AgentResult(event=ToolCallResult(status=None, message={'tool_name': 'weather_tool', 'content': 'The weather in New York City is snowing, the temperature will be below -5 degrees Celsius.', 'tool_call_id': 'call_QjTaAVBOSRt87bZfDHhWWYLk', 'metadata': None, 'timestamp': '2026-02-20T16:33:30.942121Z', 'part_kind': 'tool-return'}), event_type='ToolCallResult'

INFO:httpx:HTTP Request: POST https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"



⚾️ Agent starts to generate text:
纽约现在正在下雪，气温在零下5度以下。建议你穿厚羽绒服、帽子、围巾和手套，注意保暖哦！如果需要外出，记得防滑，安全第一。

=== Test name query ===

=== Running query: My name is wanghuan, 32 years old. ===


INFO:httpx:HTTP Request: POST https://azure-m7byy3cl-eastus2.cognitiveservices.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:root:呃, 稍等我想下啊。



⚾️ New Event:
AgentResult(event=ToolCallsOutputStart(status=None, message={'tool_name': 'final_result', 'args': '', 'tool_call_id': 'call_Tf3rFfp7fS9EL6b9l1nYXtt6', 'id': None, 'provider_details': None, 'part_kind': 'tool-call'}), event_type='ToolCallsOutputStart', finish_reason='', last_agent_name='')

⚾️ Found structured output:
name='wanghuan' age=32

⚾️ I want to transfer to the next step.
